# Request API and restructure it in pandas dataframe
An Application Programming interface [API](https://www.ibm.com/think/topics/api) is a protocol (with a set of rules) that enables a communication between software applications in order for them to exchange data. The communication is a set of requests and responses between a client and a server. Pythons [Request](https://requests.readthedocs.io/en/latest/) library allows HTTP/1.1 requests

In [1]:
import requests
import json
import pandas as pd
from IPython.display import display

The Norwegian Water Resources and Energy Directorate (Norges vassdrags- og energidirektorat - NVE) regulates the country's water resources and energy supply. One of theire databses where they offer API is the hydropower database with information on developed power plants, pumped storage power plants and pumps, including power plants that are temporarily out of operation. The database is updated continuously.

In [2]:
url = ' https://api.nve.no/web/Powerplant/GetHydroPowerPlantsInOperation'
#url = ' https://api.nve.no/web/Powerplant/'
response = requests.get(url)

data = response.json()
response.text
df=pd.DataFrame(data)

Using pythons display function to show the 5 first rows and 8 first columns:

In [3]:
display(df.iloc[:5, :8])

,VannKraftverkID,Navn,VannKVType,VannKVTypeID,HovedEier,HovedEier_OrgNr,Fylke,FylkesNr
0,2,Adamselv,Kraftverk,K,STATKRAFT ENERGI AS,987059729,Finnmark,56
1,1268,Aga,Kraftverk,K,,,Vestland,46
2,1527,Aklestad,Kraftverk,K,HÅVARD AKLESTAD,988117722,Møre og Romsdal,15
3,1677,Akslandselva,Kraftverk,K,AKSLANDSELVA KRAFTVERK AS,991005625,Vestland,46
4,2054,Ala,Kraftverk,K,SKAGERAK KRAFT AS,979563531,Innlandet,34


By using pythons list function we can list all the column headers in the dataframe:

In [4]:
print(list(df))

['VannKraftverkID', 'Navn', 'VannKVType', 'VannKVTypeID', 'HovedEier', 'HovedEier_OrgNr', 'Fylke', 'FylkesNr', 'Kommune', 'KommuneNr', 'ForsteUtnyttelseAvFalletDato', 'DatoForEldsteKraftproduserendeDel', 'MaksYtelse', 'MidProd_91_20', 'BruttoFallhoyde_M', 'Slukeevne', 'EnEkv', 'ElspotomraadeNummer', 'RegineNr', 'ErIDrift', 'IDriftDato', 'Konsesjoner', 'Kraftverkstatus', 'NVEOmraadeID', 'NVEOmraadeNavn', 'Nedborsfeltnavn', 'SPPunkt', 'SPSone', 'UnderBygging', 'UteAvDrift', 'VassdragsOmraadeID', 'VassdragsOmraadeNavn']


Here we are interested in the installed power (MW) given in the column *MaksYtelse*. Name of the Hydropower station is given in column *Navn*. We are also interested in the owner, *HovedEier*, of the power station. *MidProd_91_10* is verage annual power production in GWh/year, referenced to the inflow period 1991-2020. *BruttoFallhoyde_M* is the height difference between the water level in the water intake and the turbine/outlet elevation in meters. The head is referenced to a reservoir filling of 67%. *Slukeevne* is the maximum water flow that the turbines in a hydroelectric power plant can utilize. Expressed in m³/s. The energy equivalent, *EnEkv*, tells how much energy can be extracted per cubic meter of water through the power plant. It is expressed in kWh/m3.

In [5]:
# Define the threshold value
threshold = 370

# Filter rows where 'Score' is greater than 90 and select 'Name' and 'City' columns
filtered_df = df.loc[df['MaksYtelse'] > threshold][['Navn', 'HovedEier', 'MaksYtelse', 'MidProd_91_20','BruttoFallhoyde_M', 'Slukeevne', 'EnEkv']].sort_values(by='MaksYtelse', ascending=False)

print(display(filtered_df))

,Navn,HovedEier,MaksYtelse,MidProd_91_20,BruttoFallhoyde_M,Slukeevne,EnEkv
799,Kvilldal,STATKRAFT ENERGI AS,1240.0,3230.895,536.50,263.33,1.307
1558,Tonstad,SIRA KVINA KRAFTSELSKAP,960.0,4057.575,442.00,253.24,1.052
22,Aurland 1,HAFSLUND KRAFT AS,840.0,2099.408,850.00,112.50,2.074
1263,Saurdal,STATKRAFT ENERGI AS,640.0,1063.719,437.00,173.27,1.026
1484,Sy-Sima,STATKRAFT ENERGI AS,620.0,1637.126,905.00,80.00,2.153
1462,Svartisen,STATKRAFT ENERGI AS,600.0,2424.145,543.00,120.00,1.389
831,Lang-Sima,STATKRAFT ENERGI AS,500.0,1252.469,1065.00,51.70,2.686
1136,Rana,STATKRAFT ENERGI AS,500.0,2223.737,505.29,115.73,1.200
1555,Tokke,STATKRAFT ENERGI AS,430.0,2396.042,393.50,128.02,0.933
1617,Tyin,HYDRO ENERGI AS,374.0,1462.391,1040.00,40.00,2.597


None


Filter for rows on main owner, 'HovedEier', is Hydro Energi AS

In [6]:

filtered_df = df.loc[df['HovedEier'] == 'HYDRO ENERGI AS'][['Navn', 'HovedEier', 'MaksYtelse', 'ElspotomraadeNummer']]
display(filtered_df)

,Navn,HovedEier,MaksYtelse,ElspotomraadeNummer
302,Fivlemyr,HYDRO ENERGI AS,2.00,5
369,Frøystul,HYDRO ENERGI AS,45.60,2
552,Herva,HYDRO ENERGI AS,33.00,5
589,Holsbru,HYDRO ENERGI AS,48.90,5
916,Mannsberg,HYDRO ENERGI AS,3.52,5
960,Moflåt,HYDRO ENERGI AS,30.00,2
991,Mæl,HYDRO ENERGI AS,37.50,2
1295,Skagen,HYDRO ENERGI AS,270.00,5
1475,Svelgfoss,HYDRO ENERGI AS,92.00,2
1513,Såheim,HYDRO ENERGI AS,189.00,2


Same filter as before for rows on main owner, 'HovedEier', is Hydro Energi AS, and a specific presipitation area

In [7]:
filtered_df = df[(df['HovedEier'] == 'HYDRO ENERGI AS') & (df['Nedborsfeltnavn'] == 'Skiensvassdraget')][['Navn', 'HovedEier', 'MaksYtelse', 'ElspotomraadeNummer','Nedborsfeltnavn']]
display(filtered_df)

,Navn,HovedEier,MaksYtelse,ElspotomraadeNummer,Nedborsfeltnavn
369,Frøystul,HYDRO ENERGI AS,45.6,2,Skiensvassdraget
960,Moflåt,HYDRO ENERGI AS,30.0,2,Skiensvassdraget
991,Mæl,HYDRO ENERGI AS,37.5,2,Skiensvassdraget
1475,Svelgfoss,HYDRO ENERGI AS,92.0,2,Skiensvassdraget
1513,Såheim,HYDRO ENERGI AS,189.0,2,Skiensvassdraget
1699,Vemork,HYDRO ENERGI AS,204.0,2,Skiensvassdraget


### References

1. https://pypi.org/project/jupyter-to-medium/
2. https://www.ibm.com/think/topics/api
3. https://requests.readthedocs.io/en/latest/
4. https://www.nve.no/om-nve/aapne-data-og-api-fra-nve/